In [1]:
using IntervalArithmetic
using AutoDiff
using PurityIntervals
using KrawczykMethod2D

Syntax: krawczyk2d(function, [Interval(lo, hi), Interval(lo, hi)], precision [default is 64])


    +(AutoDiff.Ad, Any) at /home/kriukov/main/work/mexico/unam/interval-methods/modules/AutoDiff.jl:39
is ambiguous with: 
    +(Any, IntervalArithmetic.IntUnion) at /home/kriukov/main/work/mexico/unam/interval-methods/modules/IntervalArithmetic.jl:989.
To fix, define 
    +(AutoDiff.Ad, IntervalArithmetic.IntUnion)
before the new definition.
    +(Any, AutoDiff.Ad) at /home/kriukov/main/work/mexico/unam/interval-methods/modules/AutoDiff.jl:40
is ambiguous with: 
    +(IntervalArithmetic.IntUnion, Any) at /home/kriukov/main/work/mexico/unam/interval-methods/modules/IntervalArithmetic.jl:988.
To fix, define 
    +(IntervalArithmetic.IntUnion, AutoDiff.Ad)
before the new definition.
    -(AutoDiff.Ad, Any) at /home/kriukov/main/work/mexico/unam/interval-methods/modules/AutoDiff.jl:39
is ambiguous with: 
    -(Any, IntervalArithmetic.IntUnion) at /home/kriukov/main/work/mexico/unam/interval-methods/modules/IntervalArithmetic.jl:992.
To fix, define 
    -(AutoDiff.Ad, IntervalArithmetic.In

In [18]:
using PyPlot
using PyCall
@pyimport matplotlib.patches as patches
rectangle = patches.Rectangle
function draw_rectangle(x, y, xwidth, ywidth, color="grey")
    ax = gca()
    ax[:add_patch](rectangle((x, y), xwidth, ywidth, facecolor=color, alpha=0.5, linewidth=0))
end

LoadError: LoadError: InitError: UndefVarError: PyString_FromStringAndSize not defined
during initialization of module PyPlot
while loading In[18], in expression starting on line 1

In [3]:
function table(c)
    N = length(c)
    r = zeros(prec, N, N)
    a = zeros(prec, N, N)
    for i = 1:N
        for j = 1:N
            r[i, j] = sqrt((c[i][1] - c[j][1])^2 + (c[i][2] - c[j][2])^2)
            a[i, j] = atan2((c[j] - c[i])[2], (c[j] - c[i])[1])
        end
    end
    r, a
end

table (generic function with 1 method)

In [4]:
function T0(x, c, n, m)
	ω, θ = x
	#println("Argument into T0 with $n, $m: $x")
	r_nm, a_nm = table(c)
	
	ω_next = ω - r_nm[n, m]*(ω*cos(θ - a_nm[n, m]) + √(1 - ω^2)*sin(θ - a_nm[n, m]))
	θ_next = mod(θ + prec(pi) + asin(ω) + asin(ω_next), 2π)

    [ω_next, θ_next]
end

function path_general(x, c, n)
    for i = 1:length(n)-1
        if n[i] != n[i+1]
            x = T0(x, c, n[i], n[i+1])
        else
            error("Cannot hit the same disk twice in succession")
        end
    end
    x
end

path_general (generic function with 1 method)

In [5]:
function draw_phase_space(c, n, rect, tol)
    points, purities = plot_band_bisection(x -> path_general(x, c, n), rect, tol)
    for i = 1:length(points)
        if purities[i] == 0
            draw_rectangle(points[i][2].lo, points[i][1].lo, diam(points[i][2]), diam(points[i][1]), "green")
        elseif purities[i] == 1
            draw_rectangle(points[i][2].lo, points[i][1].lo, diam(points[i][2]), diam(points[i][1]), "blue")
        end
    end
    axis([0, pi/3, -1, 1])
end

draw_phase_space (generic function with 1 method)

In [6]:
function find_periodic_orbits(c, n, rect, prec, tol)
   f(x) = path_general(x, c, n)
   #f = @anon x -> path_general(x, c, n) - x
   krawczyk2d_purity_periodic(f, rect, prec, tol)
   #krawczyk2d_purity(f, rect, prec, tol)
end

find_periodic_orbits (generic function with 1 method)

In [14]:
function plot_band_bisection(f, rect, tol)
    limitrect(rect) = max(diam(rect)[1], diam(rect)[2]) < tol
    p = purity(f, rect)
    if p != -1
        if p == 1
            #println("Went to p=1")
            push!(points, rect)
            push!(purities, p)
            #println("Clean recorded")
        elseif p == 0
            #println("Went to p=0")
            if limitrect(rect)
                push!(points, rect)
                push!(purities, p)
                #println("Unclean recorded")
            else
                pieces = bisect(rect)
                for i = 1:4
                    plot_band_bisection(f, pieces[i], tol)
                end
            end
        end
    end
    points, purities
end

plot_band_bisection (generic function with 1 method)

In [7]:
# Draw phase space from n-th disk to all other - here "n" is a number

function draw_phase_space_general(c, n, rect, tol)
    points = Array{MultiDimInterval, 1}[]
    purities = Array{Int, 1}[]
    for i = 1:length(c)
        if i != n
            points_i, purities_i = plot_band_bisection(x -> path_general(x, c, [n, i]) - x, rect, tol)
            push!(points, points_i)
            push!(purities, purities_i)
        end
    end
    
    for i = 1:length(points)
        for j = 1:length(points[i])
            if purities[i][j] == 0
                draw_rectangle(points[i][j][2].lo, points[i][j][1].lo, diam(points[i][j][2]), diam(points[i][j][1]), "green")
            elseif purities[i][j] == 1
                draw_rectangle(points[i][j][2].lo, points[i][j][1].lo, diam(points[i][j][2]), diam(points[i][j][1]), "blue")
            end
        end
    end

    axis([0, pi/3, -1, 1])
    
end

draw_phase_space_general (generic function with 1 method)

In [8]:
# Symmetric 3-disk scatterer with variable separation r0
function symmetric3(r0)
    c = Array{prec, 1}[]
    push!(c, [0, 0], [r0, 0], [r0/2, r0*sqrt(3)/2]) 
    c
end

symmetric3 (generic function with 1 method)

In [9]:
c0 = symmetric3(6)

3-element Array{Array{BigFloat,1},1}:
 BigFloat[0.000000000000000000000000000000000000000000000000000000000000000000000000000000,0.000000000000000000000000000000000000000000000000000000000000000000000000000000]
 BigFloat[6.000000000000000000000000000000000000000000000000000000000000000000000000000000,0.000000000000000000000000000000000000000000000000000000000000000000000000000000]
 BigFloat[3.000000000000000000000000000000000000000000000000000000000000000000000000000000,5.196152422706632023619022220373153686523437500000000000000000000000000000000000]

In [16]:
rect = [Interval(-0.999, 0.999), Interval(0.001, big(pi)/3-0.001)]
points = Array{Interval, 1}[]
purities = Int[]

0-element Array{Int32,1}

In [19]:
draw_phase_space_general(c0, 1, rect, 1e-2)

LoadError: LoadError: UndefVarError: draw_rectangle not defined
while loading In[19], in expression starting on line 1

In [20]:
find_periodic_orbits(c0, [1, 2, 3, 1], rect, 64, 1e-4)

LoadError: LoadError: InterruptException:
while loading In[20], in expression starting on line 1